## <p style="text-align:center" color="red"><span style="color:red">Youtube Video Transcriptor - updated</span></p>



<table align="center">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/labrijisaad/Youtube-video-transcriptor"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

- **`This script is meant to be run in google colaboratory!`**
- In order to optimize the transcription time, I updated the script using python threads 😋
- This script can sometimes not detect text correctly, it's mainly due to noises or the way we speak in the video (speaking too fast or too slow)
- The general idea summary model is a community model available at [Huggingface](https://huggingface.co/) , it can sometimes get the general idea wrong, especially if there is a lack of data.


> 🙌 Notebook made by [@labriji_saad](https://github.com/labrijisaad)

### Installing the requirements

In [34]:
%%capture 

!pip install --upgrade yt_dlp
!pip install pydub SpeechRecognition ffmpeg ffprobe googletrans==3.1.0a0 transformers

In [35]:
import yt_dlp
import time
import re
import os
from pydub import AudioSegment
import speech_recognition as sr
import math
from tqdm import tqdm
from googletrans import Translator
from threading import Thread

### Downloading the audio (`url = video_link`)
> - Specify here the link of the video you want to transcribe.

In [36]:
url = "https://www.youtube.com/watch?v=IErvIekMD3U&t=1545s&ab_channel=Mrwhosetheboss"

In [37]:
ydl_opts={}

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info_dict = ydl.extract_info(url, download=False)
video_title = info_dict['title']
video_name = re.sub('[\\\\/*?:"<>|]', '', video_title)
name = video_name
ydl_opts = {
     'format': 'm4a/bestaudio/best',
         'noplaylist': True,
         'continue_dl': True,
         'outtmpl': f'./{name}.wav',
         'postprocessors': [{
             'key': 'FFmpegExtractAudio',
             'preferredcodec': 'wav',
             'preferredquality': '192',
         }],
         'geobypass':True,
         'ffmpeg_location':'/usr/bin/ffmpeg'
 }

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
     error_code = ydl.download(url)

[youtube] IErvIekMD3U: Downloading webpage
[youtube] IErvIekMD3U: Downloading android player API JSON
[youtube] IErvIekMD3U: Downloading webpage
[youtube] IErvIekMD3U: Downloading android player API JSON
[info] IErvIekMD3U: Downloading 1 format(s): 140
[download] Destination: ./19 TOXIC Tech Fails that will last Forever..wav
[download] 100% of 27.07MiB in 00:00                   
[FixupM4a] Correcting container of "./19 TOXIC Tech Fails that will last Forever..wav"
[ExtractAudio] Destination: ./19 TOXIC Tech Fails that will last Forever..wav
Deleting original file ./19 TOXIC Tech Fails that will last Forever..orig.wav (pass -k to keep)


### Spliting the audio (`min_per_split = 1`)
> - When it comes to using the free version of Google's transcriber (**`speech_recognition`**), there is a limit on the length of the video (or audio) that should not be exceeded (this limit is around 5 minutes ). To remedy this problem, the following script splits the video into one minute long intervals and puts the generated mini-videos in a directory that has a name in the form of **`split files for: Video_Name.wav`**

In [38]:
class SplitWavAudioMubin():
    def __init__(self, folder, filename):
        self.folder = folder
        self.filename = filename
        self.filepath = folder + filename
        self.audio = AudioSegment.from_wav(self.filepath)
    
    def get_duration(self):
        return self.audio.duration_seconds
    
    def single_split(self, from_min, to_min, split_filename):
        t1 = from_min * 60 * 1000
        t2 = to_min * 60 * 1000
        split_audio = self.audio[t1:t2]
        split_audio.export(split_filename, format="wav")
        
    def multiple_split(self, min_per_split):
        total_mins = math.ceil(self.get_duration() / 60)
        for i in range(0, total_mins, min_per_split):
            split_fn = str(i) + '_' + self.filename
            self.single_split(i, i+min_per_split, split_fn)
            if i == total_mins - min_per_split:
                print('All splited successfully')
        print('>>> Video duration: ' + str(self.get_duration()))
        
def split_audio(file_name):
    directory = "splitted files for: " + file_name
    os.mkdir(directory)
    os.chdir(directory)
    split_wav = SplitWavAudioMubin("../", file_name)
    split_wav.multiple_split(min_per_split=1)

In [39]:
file_name = "{}.wav".format(video_name)
split_audio(file_name)

All splited successfully
>>> Video duration: 1754.128253968254


### Recognizing the text (` language = "en-US"`) https://cloud.google.com/speech-to-text/docs/languages
> - To perform text detection, we must first specify the language spoken in the video. To do this, we must search for the keyword equivalent to language in the language catalog available in the link on the title. ( In our case, it's **`English`** so the keyword is **`en-US`** )

In [40]:
search_dir = "./"
files = filter(os.path.isfile, os.listdir(search_dir))
files = [os.path.join(search_dir, f) for f in files] # add path to each file
files.sort(key=lambda x: os.path.getmtime(x))
files

['./0_19 TOXIC Tech Fails that will last Forever..wav',
 './1_19 TOXIC Tech Fails that will last Forever..wav',
 './2_19 TOXIC Tech Fails that will last Forever..wav',
 './3_19 TOXIC Tech Fails that will last Forever..wav',
 './4_19 TOXIC Tech Fails that will last Forever..wav',
 './5_19 TOXIC Tech Fails that will last Forever..wav',
 './6_19 TOXIC Tech Fails that will last Forever..wav',
 './7_19 TOXIC Tech Fails that will last Forever..wav',
 './8_19 TOXIC Tech Fails that will last Forever..wav',
 './9_19 TOXIC Tech Fails that will last Forever..wav',
 './10_19 TOXIC Tech Fails that will last Forever..wav',
 './11_19 TOXIC Tech Fails that will last Forever..wav',
 './12_19 TOXIC Tech Fails that will last Forever..wav',
 './13_19 TOXIC Tech Fails that will last Forever..wav',
 './14_19 TOXIC Tech Fails that will last Forever..wav',
 './15_19 TOXIC Tech Fails that will last Forever..wav',
 './16_19 TOXIC Tech Fails that will last Forever..wav',
 './17_19 TOXIC Tech Fails that will last

In [41]:
def speech_recognizer(files, frames, i):  ## This function recognizes speech in our WAV files
    texts = []
    recognizer = sr.Recognizer()

    for file in tqdm(files):
        with sr.AudioFile(file) as source:
            recorded_audio = recognizer.listen(source)
        try:
            text = recognizer.recognize_google(
                recorded_audio,
                language="en-US"  ## Replace with language keyword
            )
            texts.append(text)
        except Exception as ex:
            print(ex)
    result = ""
    for text in texts:
        result += " " + text
    frames[i] = result
    return result

def split_files(files, n_batches): ## This function split the files evenly between the threads we have.
    k, m = divmod(len(files), n_batches)
    return list(files[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n_batches))

def main(n_batches=8, verbose=True): ## By default, the maximum capacity of threads supported in collab is 8
    threads = [None]*n_batches
    frames = [None]*n_batches
    batches = split_files(files, n_batches)
    start = 0
    for i in range(len(batches)):
        if i>0:
            start_index=len(batches[i-1])
        else:
            start_index = 0
        t = Thread(target=speech_recognizer, args=(batches[i], frames, i))
        threads[i] = t
        t.start()
    for t in threads:
        t.join()
    return frames

In [42]:
if __name__ == "__main__":
    frames = main()

  0%|          | 0/4 [00:00<?, ?it/s]


  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]




  0%|          | 0/3 [00:00<?, ?it/s]



  0%|          | 0/4 [00:00<?, ?it/s]





  0%|          | 0/4 [00:00<?, ?it/s]






 25%|██▌       | 1/4 [00:07<00:23,  7.79s/it]





 25%|██▌       | 1/4 [00:07<00:22,  7.56s/it]






 33%|███▎      | 1/3 [00:07<00:15,  7.89s/it]


 25%|██▌       | 1/4 [00:08<00:26,  8.75s/it]




 33%|███▎      | 1/3 [00:08<00:17,  8.85s/it]

 25%|██▌       | 1/4 [00:08<00:26,  8.95s/it]



 25%|██▌       | 1/4 [00:09<00:27,  9.28s/it]






 67%|██████▋   | 2/3 [00:15<00:07,  7.54s/it]

 50%|█████     | 2/4 [00:16<00:16,  8.14s/it]


 50%|█████     | 2/4 [00:16<00:16,  8.16s/it]





 50%|█████     | 2/4 [00:16<00:16,  8.23s/it]




 67%|██████▋   | 2/3 [00:16<00:08,  8.37s/it]






100%|██████████| 3/3 [00:17<00:00,  5.97s/it]




 75%|███████▌  | 3/4 [00:23<00:08,  8.40s/it]


 75%|███████▌  | 3/4 [00:23<00:07,  7.71s/it]

 75%|████

[" prepare yourself in 19 old-time from space of bad ideas that's completely misunderstood readers like what kind of newspaper in the morning but then think oh wait no bikes that you could offer a complete t3micro by movement and one that you be able to stick to any surface you wanted extra special rubberized material on the back they come up with feel like these highly optimized human beings by bulky cumbersome glasses temples is", " everyone dream watch my videos right in the beginning you might know that I built this by filming on an Ikea table for simple reason they have a business model is that what issue was the Ikea forgot the simple fact that hair dryers blow out hot air and the plastic melts see where I'm going with this don't say that the attack wasn't clever essentially disclosure has an ultrasound speaker inside of it which comparative Soundwave you mount them", " quite quickly lost interest in spending time with buried in the cupboard 10sp Enright your phone is the dumbest

### Saving the recognized text
> - After performing the speech detection, we save the resulting text in a file which is in the form of **`Transcription_Video_Name.txt`**

In [45]:
result = ""
for text in frames:
    result += " " + text

os.chdir("../")
text_file = open("Transcription_"+ file_name[:-4] +".txt", "w")
text_file.write(result)
text_file.close()

In [ ]:
result

### Translating the recognized text (`dest='fr'`)
> - In addition, we have tried here to translate the text into French after transcription, using the Google API (**`googletrans`**)
> - To correctly use this API, we must replace the dest variable with the output language keyword ( In our case, **`dest='fr'`**)

In [46]:
translator = Translator()

translate_text = ""
for text in frames:
    translate_text += " " + translator.translate(text, dest='fr').text
print(translate_text)

 préparez-vous à 19 ans à partir de l'espace des mauvaises idées c'est des lecteurs complètement incompris comme quel genre de journal le matin mais ensuite pensez oh attendez non vélos que vous pourriez offrir un t3micro complet par mouvement et que vous puissiez coller sur n'importe quelle surface que vous vouliez matériau caoutchouté spécial supplémentaire à l'arrière ils viennent avec l'impression que ces êtres humains hautement optimisés par des tempes de lunettes encombrantes et encombrantes sont tout le monde rêve regardez mes vidéos dès le début, vous savez peut-être que j'ai construit cela en filmant sur une table Ikea pour la simple raison qu'ils ont un modèle commercial, c'est que quel problème était l'Ikea ​​a oublié le simple fait que les sèche-cheveux soufflent de l'air chaud et que le plastique fond voir où je veux en venir ne dites pas que l'attaque n'était pas intelligente, essentiellement la divulgation a un haut-parleur à ultrasons à l'intérieur de laquelle Soundwave

### Saving the translated text
> - We save the resulting translated text in a file which is in the form of **`Transcription_translated_Video_Name.txt`**

In [47]:
text_file = open("Transcription_translated_"+ file_name[:-4] +".txt", "w")
text_file.write(translate_text)
text_file.close()

### General Idea summarization
> - Finally, we can use the text we have recovered to have a summary of the general idea discussed in the video
> - Here it is necessary to specify the **`max_length`** and the **`min_length`**, by default we have chosen that the length of the general idea of a text must be at least 10% of the total length of the text.

In [48]:
from transformers import pipeline
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

In [49]:
ARTICLE = result
summary_text = summarizer(ARTICLE, max_length=100, min_length=int(len(result.split(" "))/10), do_sample=False)[0]["summary_text"]
print(summary_text)

I built this by filming on an Ikea table for simple reason they have a business model is that what issue was the Ikea forgot the simple fact that hair dryers blow out hot air and the plastic melts. Microsoft correctly predicted the gaming with a huge and growing markets. Mike mattock fails that videotape biggest text that you've ever seen your entire life.


> - 🙌 Notebook made by [@labriji_saad](https://github.com/labrijisaad)
> - 🔗 Linledin [@labriji_saad](https://www.linkedin.com/in/labrijisaad/)